In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
import csv

In [ ]:
## 아웃풋 파일 경로 설정

path_linked_list = "/content/gdrive/My Drive/Colab Notebooks/NetworkX/Linked List/"
path_node_info = "/content/gdrive/My Drive/Colab Notebooks/NetworkX/Node Information List/"

In [ ]:
## 네트워크 구조 생성

df_activity_list = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/NetworkX/DSME_ACTIVITY_LIST.csv", encoding='utf-8')
df_relation_list = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/NetworkX/DSME_RELATION.csv", encoding='utf-8')

df_activity_list.set_index('ACTIVITYCODE', inplace = True)

act_network = nx.DiGraph()
for idx, row in df_activity_list.iterrows():
    act_network.add_node(idx)
    for item, col in zip(row, df_activity_list.columns):
        act_network.nodes[idx][col] = item

for idx, row in df_relation_list.iterrows():
    act_network.add_edge(row['PREVACTIVITYCODE'], row['NEXTACTIVITYCODE'])
    for item, col in zip(row, df_relation_list.columns):
        if col != 'PREVACTIVITYCODE' and col != 'NEXTACTIVITYCODE':
            act_network.edges[row['PREVACTIVITYCODE'], row['NEXTACTIVITYCODE']][col] = item

In [ ]:
## 열 이름 목록

column_list = []
column_list.append('ACTIVITYCODE')
column_list.append('in_degree')
column_list.append('out_degree')
for attr in act_network.nodes['W183GT00'].keys():
    column_list.append(attr)

In [ ]:
## 끊어진 데이터 목록

with open(path_node_info + 'unlinked_activity.csv', mode='w', encoding='utf-8-sig') as unlinked_file:
    writer = csv.writer(unlinked_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(column_list)

    for subgraph in nx.weakly_connected_components(act_network):
        if len(subgraph) == 1:
            for n in subgraph:
                try:
                    temp_list = []
                    temp_list.append(n)
                    temp_list.append(act_network.in_degree(n))
                    temp_list.append(act_network.out_degree(n))
                    for attr in act_network.nodes[n].values():
                        temp_list.append(attr)
                    # print(temp_list)
                    writer.writerow(temp_list)
                    
                except:
                    pass


In [ ]:
## 각 Subgraph의 데이터 목록

len_list = []

for subgraph in nx.weakly_connected_components(act_network):
    if len(subgraph) != 1:
        if len_list.count(len(subgraph)) == 0:
            file_name = 'activity_list_' + str(len(subgraph)) + '(0)' + '.csv'
        else:    
            file_name = 'activity_list_' + str(len(subgraph)) + '(' + str(len_list.count(len(subgraph))) + ')' + '.csv'
        
        len_list.append(len(subgraph))

        with open(path_node_info + file_name, mode='w', encoding='utf-8-sig') as linked_file:
            writer = csv.writer(linked_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(column_list)

            for n in subgraph:
                try:
                    temp_list = []
                    temp_list.append(n)
                    temp_list.append(act_network.in_degree(n))
                    temp_list.append(act_network.out_degree(n))
                    for attr in act_network.nodes[n].values():
                        temp_list.append(attr)
                    # print(temp_list)
                    writer.writerow(temp_list)
                    
                except:
                    pass




In [ ]:
## 각 Subgraph의 연결리스트

len_list = []

for subgraph in nx.weakly_connected_components(act_network):
    if len(subgraph) != 1:
        if len_list.count(len(subgraph)) == 0:
            file_name = 'linked_list_' + str(len(subgraph)) + '(0)' + '.csv'
        else:    
            file_name = 'linked_list_' + str(len(subgraph)) + '(' + str(len_list.count(len(subgraph))) + ')' + '.csv'
        
        len_list.append(len(subgraph))

        with open(path_linked_list + file_name, mode='w', encoding='utf-8-sig') as linked_file:
            writer = csv.writer(linked_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

            for n in subgraph:
                try:
                    temp_list = []
                    temp_list.append(act_network.in_degree(n))
                    temp_list.append(act_network.out_degree(n))
                    temp_list.append(n)

                    linked_node_list = []
                    for edge in act_network.out_edges(n):
                        linked_node_list.append(edge[1])

                    temp_list += linked_node_list
                    writer.writerow(temp_list)
                
                except:
                    pass